In [1]:
  pip install yfinance

     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 6.4 MB 21.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
import pandas as pd
import yfinance as yf
import numpy as np

In [7]:
aapl = yf.download('AAPL', '2021-01-01', '2022-01-01')

aapl

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,128.453445,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,130.041611,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,125.664215,155088000
2021-01-07,128.360001,131.630005,127.860001,130.919998,129.952271,109578200
2021-01-08,132.429993,132.630005,130.229996,132.050003,131.073914,105158200
...,...,...,...,...,...,...
2021-12-27,177.089996,180.419998,177.070007,180.330002,180.100540,74919600
2021-12-28,180.160004,181.330002,178.529999,179.289993,179.061859,79144300
2021-12-29,179.330002,180.630005,178.139999,179.380005,179.151749,62348900


In [10]:
y = aapl['Close']
X = pd.Series(np.arange(1, 253))